<h1 style="text-align: center;">  
    Vaccine Availability Alert
</h1>

### Import Libraries

In [ ]:
from cowin_api import CoWinAPI
import pandas as pd
import time
from playsound import playsound
import datetime
import random

### Enter the required State Id
##### E.g. 9 for Delhi. Refer API Docs for all

In [ ]:
state_id = '9'
cowin = CoWinAPI()
districts = cowin.get_districts(state_id)
district_ids=districts["districts"]
dist=[]
for district in district_ids:
    dist.append(district["district_id"])
print(dist)

### Set other filters 

In [ ]:
date = '21-05-2021'  # Optional. Takes today's date by default
min_age_limit = 18  # Optional. By default returns centers without filtering by min_age_limit

### Function to fetch the required slots available from Cowin API

In [66]:
aval=[]

def getAvailableSlots(dist,date,min_age_limit):
    df = pd.DataFrame();
    for i in dist:
        available_centers = cowin.get_availability_by_district(str(i), date, min_age_limit)
        for center in available_centers["centers"]:
            df = df.append(
                {
                    "Centre Id": center["center_id"], 
                    "Center Name": center["name"], 
                    "District": center["district_name"],
                    "Pincode": center["pincode"],
                    "FeeType": center["fee_type"],
                    "Vaccine": center["sessions"][0]["vaccine"],
                    "Dose 1" : center["sessions"][0]["available_capacity_dose1"]
                }, ignore_index=True);
    filtered_df = df[(df["Dose 1"]>0) & (df["Vaccine"]=="COVAXIN")];
    return(len(filtered_df), filtered_df)

### Polling every 10 secs for slot availability

In [57]:
faces = ["( ͡° ͜ʖ ͡°)","(▀̿Ĺ̯▀̿ ̿)","¯\_(ツ)_/¯","ಠ_ಠ","(͡ ͡° ͜ つ ͡͡°)"]

In [ ]:
while(True):
    try:
        num_available_slots, available_slots = getAvailableSlots(dist,date,min_age_limit); 
        if(num_available_slots > 0):
            print(datetime.datetime.now(), available_slots);
            playsound("C:\\Users\\Shivam Dargan\\Downloads\\Red Alert Ringtone-tones6.com.mp3");
        else:
            print(datetime.datetime.now(), "Still looking... "+random.choice(faces))
        time.sleep(10)
    except Exception as e:
        print(datetime.datetime.now(), "API Unresponsive", e)
        time.sleep(20)

2021-05-21 00:22:08.035591 Still looking... ¯\_(ツ)_/¯
2021-05-21 00:22:19.097476 Still looking... (▀̿Ĺ̯▀̿ ̿)
